In this file I create:

1. The connection between PostgreSQL and Python

2. The database called **etl_workshop_first**.

3. Two tables with its properties: **raw_applicant** and **applicant**.

In [ ]:
pip install -r code/requirements.txt

Import libraries:

In [2]:
import pandas
from sqlalchemy import create_engine

from code.connect_database import Connection_Postgres

Create connection:

In [3]:
# Create connection with PostgreSQL
connection = Connection_Postgres()
# Close connection
connection.close_connection_database()